Поскольку у pymc3 какие-то проблемы с theano, на Colab пришлось поставить dev версию через github.

In [59]:
!pip install git+https://github.com/pymc-devs/pymc3 numpyro numpy pandas matplotlib seaborn arviz scipy

  Cloning https://github.com/pymc-devs/pymc3 to /tmp/pip-req-build-r6qdqfkq
  Running command git clone -q https://github.com/pymc-devs/pymc3 /tmp/pip-req-build-r6qdqfkq
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [60]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import arviz as az
sns.set()
from scipy import stats

In [62]:
import jax.numpy as jnp
import jax
from jax import random, vmap
from jax.scipy.special import logsumexp

In [63]:
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

Вспомним формулу Байеса:
$$
p(\theta | x) = \dfrac{p(x | \theta)p(\theta)}{p(x)}
$$

Простой дискретный пример.

В Небольшом пруду водятся караси и щуки, причём согласно поверью, в любой момент времени доля карасей в Небольшом пруду может равновероятно быть либо $1/3$, либо $2/3$ в зависимости от погодных условий. Исследовательница Авдотья вылавливает десять рыб подряд. Оказалось, что в полученной выборке шесть щук и четыре карася.

а) Найдите оценку максимального правдоподобия $\hat{p}_{c}$, рассуждая с позиций частотного подхода.

Ответ: $\hat{p}_{c} = 4/10$

б) Сделаем то же самое в рамках байесовского подхода. 

$$ 
p(\theta | y)  \propto p(y| \theta) \times p(\theta)
$$

$ p(\theta) = 1/2 $ (оба значения доли карасей равновероятны), $ p(y| \theta) = p^x \times (1-p)^{n-x}$. Посчитаем апостериорное распределение для обоих возможных значений $p$: 

In [64]:
p1 = (1/3) ** 4 * (2/3) ** 6 * (1/2)
p2 = (2/3) ** 4 * (1/3) ** 6 * (1/2)
print(f"Апостериорная вероятность того, что вероятность поймать карася 1/3, составляет {p1/(p1+p2)}.")
print(f"Апостериорная вероятность того, что вероятность поймать карася 2/3, составляет {p2/(p1+p2)}.")

Апостериорная вероятность того, что вероятность поймать карася 1/3, составляет 0.8.
Апостериорная вероятность того, что вероятность поймать карася 2/3, составляет 0.2.


Это вполне логично - 0.4 ближе к 1/3, чем к 2/3. 

#### Задание 1. Караси и щуки.

В Большом пруду водятся караси и щуки, причём согласно поверью, вероятность встретить карася равномерно распределена на отрезке от 0 до 1. Ловля рыбы в Большом пруду запрещена, поэтому исследовательница Авдотья решает провести численный эксперимент для вывода апостериорного распределения вероятности встретить карася.

Сгенерируйте выборку из из 100 наблюдений, каждое из которых является идентификатором того, является ли пойманная рыба карасём.

In [65]:
y = stats.bernoulli(p=0.3).rvs(100)
print('Доля карасей в выборке:', y.mean())
y

Доля карасей в выборке: 0.22


array([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

1. Задайте вероятностную модель.

In [66]:
def model(y):
  p = numpyro.sample('share', dist.Uniform(0, 1))
  y_obs = numpyro.sample('obs', dist.Bernoulli(p), obs=y)

2. Оцените модель и получите приблизительное апостериорное распределение параметров.

In [67]:
kernel = NUTS(model) # NUTS sampler 
# 100 первых наблюдений будет выкинуто при оценке апостериорного распределения
# оставшиеся 500 наблюдений будут сохранены
mcmc = numpyro.infer.MCMC(kernel, num_warmup=100, num_samples=1000)
mcmc.run(random.PRNGKey(123), y = y)
mcmc.print_summary()
samples_1 = mcmc.get_samples()

sample: 100%|██████████| 1100/1100 [00:02<00:00, 401.20it/s, 3 steps of size 8.09e-01. acc. prob=0.94]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
     share      0.22      0.04      0.22      0.17      0.30    292.31      1.00

Number of divergences: 0


In [68]:
print(f"Выборочное среднее апостериорного распределения: {samples_1['share'].mean()}.")

Выборочное среднее апостериорного распределения: 0.22409188747406006.


In [69]:
plt.figure(figsize=(9, 5));
plt.hist(samples_1['share'], bins=50, histtype='step', label='posterior distribution', color='blue');
plt.hist(np.linspace(0, 1, 500), bins=50, histtype='step', label='prior distribution', color='green');
plt.axvline(np.mean(y), color='red', label='sample estimate');
plt.legend(loc='best');


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 648x360 with 1 Axes>

У `numpyro` заметно сразу два минуса:
* Нельзя нарисовать динамику по цепям для генерируемого апостериорного распределения.
* Не считаются никакие статистики, которые говорили бы о сходимости апостериорного распределения. В этом плане `PyMC3` гораздо лучше.

Попробуем то же самое сделать через PyMC3, благо синтаксис похож.

In [70]:
ndraws = 1000
nburn = 100

with pm.Model() as model:
  p = pm.Uniform('p', lower=0, upper=1)
  y_obs = pm.Bernoulli('obs', p=p, observed=y)

  step = pm.NUTS()
  trace = pm.sample(# сколько наблюдений мы хотим сэмплировать
                    ndraws,
                    # сколько наблюдений мы хотим "сжечь"
                    tune=nburn, 
                    # удаляем ли мы "сжигаемые" набл-я из апостериорной выборки
                    discard_tuned_samples=True, 
                    # сэмплировщик
                    step=step, 
                    random_seed=np.random.seed(123),
                    # количество ядер, чтоб считать параллельно
                    cores=4,
                    # количество цепей Маркова
                    chains=4)

Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]
INFO:pymc3:NUTS: [p]


Sampling 4 chains for 100 tune and 1_000 draw iterations (400 + 4_000 draws total) took 4 seconds.
INFO:pymc3:Sampling 4 chains for 100 tune and 1_000 draw iterations (400 + 4_000 draws total) took 4 seconds.
The acceptance probability does not match the target. It is 0.9299387377087733, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9199731421955196, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8789830110049004, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9513117732351652, but should be close to 0.8. Try to increase the number of tuning steps.


In [71]:
az.plot_trace(trace);

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 864x144 with 2 Axes>

В `trace` есть много чего, при желании можно вытащить, например, выборку из апостериорного распределения для каждой цепи. 

In [72]:
plt.figure(figsize=(9, 5));
plt.hist(np.array(trace.posterior.p).ravel(), bins=50, histtype='step', label='posterior distribution', color='blue');
plt.hist(np.linspace(0, 1, 500), bins=50, histtype='step', label='prior distribution', color='green');
plt.axvline(np.mean(y), color='red', label='sample estimate');
plt.legend(loc='best');

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 648x360 with 1 Axes>

Попробуем вытащить характеристики апостериорного распределения:

In [73]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p,0.225,0.041,0.146,0.301,0.001,0.001,1646.0,1898.0,1.0


In [74]:
print(f"Выборочное среднее апостериорного распределения: {np.array(trace.posterior.p).ravel().mean()}.")

Выборочное среднее апостериорного распределения: 0.22505664330935957.


Посмотрим, удалились ли те наблюдения из апостериорного распределения, которые мы хотели сжечь. Если да, то shape у trace.posterior.p = (4, 1000).

In [75]:
np.array(trace.posterior.p).shape

(4, 1000)

#### Задание 2. Структурный сдвиг.

In [76]:
years = np.arange(1800, 2000)
casual_1 = stats.poisson(3).rvs(100)
casual_2 = stats.poisson(1).rvs(100)
casualties = np.hstack((casual_1, casual_2))

In [77]:
df = pd.DataFrame({'year': years, 'casualties': casualties})

In [78]:
plt.figure(figsize=(11, 5));
plt.plot(df.year, df.casualties, 'o', alpha=0.4, markersize=8);
plt.title('Распределение числа аварий на шахтах');
plt.axvline(1900, color='red');
plt.text(1910, 6, 'Структурный сдвиг: 1900?', bbox={'facecolor': 'white', 'pad': 8, 'edgecolor':'black'});

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 792x360 with 1 Axes>

Будем считать, что авариина шахте распределены по Пуассону и независимы:
$x_i \sim \mathrm{Pois}(\lambda)$, где

$$
\lambda = \begin{cases}
\lambda_1, t < \tau, \\
\lambda_2, t \ge \tau
\end{cases}
$$

Априорное распределение $\lambda$: $\lambda \sim \mathrm{Exp}(\alpha)$, где $alpha$ - некоторый гиперпараметр, который мы не знаем.

Оценим $\alpha$ на основе выборки:

In [79]:
y = casualties
alpha = 1 / np.mean(y)
alpha

0.4938271604938272

In [80]:
with pm.Model() as DisasterModel:
  # априорные распределения параметров
  tau = pm.DiscreteUniform("tau", lower=years.min(), upper=years.max())

  early_rate = pm.Exponential("early_rate", alpha)
  late_rate = pm.Exponential("late_rate", alpha)

  # разные значения alpha в зависимости от tau
  rate = pm.math.switch(tau>=years, early_rate, late_rate)

  # связь y c rate
  y_obs = pm.Poisson('disasters', rate, observed=y)

In [81]:
n_draws = 1000
n_burn = 100

with DisasterModel:
  sample = pm.sample(draws=n_draws,
                     tune=n_burn,
                     chains=4, 
                     cores=4, 
                     random_seed=np.random.seed(123))

Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
INFO:pymc3:CompoundStep
>Metropolis: [tau]
INFO:pymc3:>Metropolis: [tau]
>NUTS: [early_rate, late_rate]
INFO:pymc3:>NUTS: [early_rate, late_rate]


Sampling 4 chains for 100 tune and 1_000 draw iterations (400 + 4_000 draws total) took 7 seconds.
INFO:pymc3:Sampling 4 chains for 100 tune and 1_000 draw iterations (400 + 4_000 draws total) took 7 seconds.
The acceptance probability does not match the target. It is 0.923206067504259, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9352360591278218, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.931945273866517, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9394531573518786, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.
ERROR:pymc3:The estimated number of effective samples is smaller than 200 for some parameters.


In [82]:
az.plot_trace(sample);

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 864x432 with 6 Axes>

In [83]:
az.summary(sample)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
tau,1898.554,1.956,1894.000,1901.000,0.255,0.181,86.0,73.0,1.04
early_rate,3.016,0.176,2.668,3.328,0.003,0.002,2781.0,2630.0,1.00
late_rate,1.054,0.107,0.844,1.243,0.004,0.003,690.0,1332.0,1.01


In [84]:
plt.figure(figsize=(7, 5));
plt.hist(np.array(sample.posterior.tau).ravel(), bins=50);
plt.title('Распределение значения года для структурного сдвига');


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 504x360 with 1 Axes>

Оценим вероятность того, что $\lambda_i$ различны:

In [85]:
np.mean(np.array(sample.posterior.early_rate) - \
        np.array(sample.posterior.late_rate) > 0)

1.0

### Байесовская линейная регрессия.

Набор данных - WaffleDivorce, содержащий информацию о проценте разводов в 50 штатах США.

In [86]:
DATASET_URL = 'https://raw.githubusercontent.com/rmcelreath/rethinking/master/data/WaffleDivorce.csv'
df = pd.read_csv(DATASET_URL, sep=';')
df.head(5)

,Location,Loc,Population,MedianAgeMarriage,Marriage,Marriage SE,Divorce,Divorce SE,WaffleHouses,South,Slaves1860,Population1860,PropSlaves1860
0,Alabama,AL,4.78,25.3,20.2,1.27,12.7,0.79,128,1,435080,964201,0.45
1,Alaska,AK,0.71,25.2,26.0,2.93,12.5,2.05,0,0,0,0,0.00
2,Arizona,AZ,6.33,25.8,20.3,0.98,10.8,0.74,18,0,0,0,0.00
3,Arkansas,AR,2.92,24.3,26.4,1.70,13.5,1.22,41,1,111115,435450,0.26
4,California,CA,37.25,26.8,19.1,0.39,8.0,0.24,0,0,0,379994,0.00


Нам предлагаются следующие признаки для анализа:

* `Population`
* `MedianAgeMarriage`
* `Marriage`
* `WaffleHouses`
* `South`.

In [87]:
regr = ['Divorce', 'Population', 'MedianAgeMarriage', 'Marriage',
        'WaffleHouses', 'South']
sns.pairplot(df, x_vars=regr, y_vars=regr);

RuntimeError: ignored

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f2e0599d560> (for post_execute):


RuntimeError: ignored

RuntimeError: ignored

<Figure size 1080x1080 with 36 Axes>

Хорошая корреляция у `Divorce` есть только с `MedianAgeMarriage` и `Marriage`, и ещё слабая с `WaffleHouses`.

In [ ]:
sns.regplot('WaffleHouses', 'Divorce', df);

Скорее всего, корреляция с числом ресторанов в штате ложная - вряд ли этот фактор как-то влияет на процент разводов.

In [ ]:
X = df[['Divorce', 'Marriage', 'MedianAgeMarriage']]

In [ ]:
stand = lambda x: (x-x.mean()) / x.std()
X = X.apply(stand)

Модель:

$$ \mathrm{divorce} \sim \mathcal{N}(\mu, \sigma^2). $$$$ \mu = \beta_0 + \beta_1\mathrm{marriage} + \beta_2\mathrm{age}. $$

Иначе говоря, мы подразумеваем нормальность ошибки.

In [ ]:
#  случайными у нас будут коэффициенты, три переменные и сигма
def model(divorce=None, marriage=None, age=None):
  beta0 = numpyro.sample('beta0', dist.Normal(0, 0.2))
  MARRIAGE, AGE = 0, 0

  # ифы сделаны, чтобы можно было добавлять разные регрессоры в модель
  if marriage is not None:
    beta1 = numpyro.sample('beta1', dist.Normal(0, 0.5))
    MARRIAGE = beta1 * marriage

  if age is not None:
    beta2 = numpyro.sample('beta2', dist.Normal(0, 0.5))
    AGE = beta2 * age

  sigma = numpyro.sample('sigma', dist.Exponential(1))
  mu = beta0 + MARRIAGE + AGE
  numpyro.sample('obs', dist.Normal(mu, sigma), obs=divorce)

Используем в качестве регрессора только `marriage`:

In [ ]:
kernel = NUTS(model)
mcmc = MCMC(sampler=kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(random.PRNGKey(0), 
         marriage=X['Marriage'].values,
         divorce=X['Divorce'].values)
mcmc.print_summary()
samples_1 = mcmc.get_samples()

In [ ]:
trace_df = pd.DataFrame({'beta0': np.array(samples_1['beta0']),
                         'beta1': np.array(samples_1['beta1']),
                         'sigma': np.array(samples_1['sigma'])})

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

ax[0].hist(trace_df.beta0, bins=50, color='blue')
ax[0].set_title('Распределение beta_0')
ax[1].hist(trace_df.beta1, bins=50, color='green')
ax[1].set_title('Распределение beta_1')
ax[2].hist(trace_df.sigma, bins=50, color='orange')
ax[2].set_title('Распределение sigma')

fig.show();

In [ ]:
pd.Series(np.array(samples_1['sigma'])).plot(kind='hist',
                                             bins=50, 
                                             title='Распределение sigma');